In [1]:
from copy import deepcopy
from sympy import sympify

In [ ]:
with open("input.txt") as f:
    lines = f.readlines()
lines = [l.strip() for l in lines]
lines[:5]

In [3]:
start_vals = {}
equations = []
first = True
for line in lines:
    if len(line) == 0:
        first = False
        continue

    if first:
        spl = line.split(":")
        start_vals[spl[0]] = bool(int(spl[1].strip()))
    else:
        spl = [s.strip() for s in line.split("->")]
        res = spl[1]
        eq = spl[0].split()
        variables = [eq[0], eq[2]]
        operator = eq[1]
        equations.append(
            {
                "variables": variables,
                "operator": operator,
                "result": res,
                "solved": False,
            }
        )

# equations

In [ ]:
values = deepcopy(start_vals)
open_equations = deepcopy(equations)
# Replace newly computed values in equations until all are solved
while open_equations:
    for eq in open_equations:
        if all([v in values for v in eq["variables"]]):
            # print(eq)
            if eq["operator"] == "AND":
                values[eq["result"]] = (
                    values[eq["variables"][0]] and values[eq["variables"][1]]
                )
                eq["solved"] = True
            elif eq["operator"] == "OR":
                values[eq["result"]] = (
                    values[eq["variables"][0]] or values[eq["variables"][1]]
                )
                eq["solved"] = True
            elif eq["operator"] == "XOR":
                values[eq["result"]] = (
                    values[eq["variables"][0]] != values[eq["variables"][1]]
                )
                eq["solved"] = True
    open_equations = [e for e in open_equations if not e["solved"]]

# values
z_outputs = sorted([(k, v) for k, v in values.items() if k.startswith("z")])
binary_output = "".join(reversed(["1" if b[1] else "0" for b in z_outputs]))
print(binary_output)
print(int(binary_output, 2))

In [5]:
# part 2

# The idea is to simplify the equations as they are all the same on every level just with different numbers.
# And then by hand search the needed replacements using the printed output
# This means we need to resolve each equation resulting in zXX to only base vars
# And then print it somewhat sorted. As I did not want to implement that sorting part, I use sympy, read the equation and then print it
# This prints it sorted
# Then I replace all the carry overs recursively from start up to the latest equation
# It is easy to spot the first problematic equation in the output
# Then look in the output for something that resembles what we need
# Equations always need to start like this z05 = x05 ^ y05 ^ ((x04 & y04) | (c04))
# Then use the code snippets below to search for nodes that match this pattern and switch them and watch the output again
# Not beautiful, but works :)

# My first problematic line is z11 = x11 & y11
# So I search for an equation that has x11 XOR y11
# And then look at the parent of that that also uses an xor
# And then I switch those 2

In [ ]:
# Resolve an equation to only base variables
def resolve_eq(res, equations):
    if res.startswith("x") or res.startswith("y"):
        return f"{res}"

    eq = [eq for eq in equations if eq["result"] == res][0]
    vrs = sorted(eq["variables"])
    return f'({resolve_eq(vrs[0], equations)} {eq["operator"]} {resolve_eq(vrs[1], equations)})'


resolve_eq("z05", equations)

In [ ]:
# Use this to search for nodes
looking_for_var = "x11"
looking_for_op = "XOR"
[
    e
    for e in equations
    if e["operator"] == looking_for_op
    and (e["variables"][0] == looking_for_var or e["variables"][1] == looking_for_var)
]

In [ ]:
looking_for_var = "vbj"
[
    e
    for e in equations
    if (e["variables"][0] == looking_for_var or e["variables"][1] == looking_for_var)
]
# This returns 2, but we are only interested in the XOR one

In [ ]:
def switch_wires(w1, w2, equations):
    e1 = [e for e in equations if e["result"] == w1][0]
    e2 = [e for e in equations if e["result"] == w2][0]

    e1["result"] = w2
    e2["result"] = w1


str_eqs = []
open_equations = deepcopy(equations)

switch_wires("vkq", "z11", open_equations)
# Removing last 3 switches to not leak too much
# switch_wires("x", "x", open_equations)
# switch_wires("x", "x", open_equations)
# switch_wires("x", "x", open_equations)

for eq in open_equations:
    if eq["result"].startswith("z"):
        str_eqs.append(f"{eq["result"]} = {resolve_eq(eq["result"], open_equations)}")

eq_str_list = []
for eq in sorted(str_eqs):
    e = eq.split(" = ")[1]
    expr = sympify(
        e.replace("AND", "&").replace("XOR", "^").replace("OR", "|").replace(" ", ""),
        convert_xor=False,
    )
    str_expr = str(expr)
    eq_str_list.append(f"{eq.split(" = ")[0]} = {str_expr}")


# replace carry overs
eq_str = "\n".join(eq_str_list)

repl_eq_str = eq_str.replace("x00 & y00 & (x01 ^ y01)", "c01")
for i in range(1, 45):
    repl_eq_str = repl_eq_str.replace(
        f"(x{i+1:02} ^ y{i+1:02}) & ((x{i:02} & y{i:02}) | (c{i:02}))", f"c{i+1:02}"
    )

print(repl_eq_str)

In [10]:
# ",".join(sorted([<solution list>]))